# Plot augmented Nek temperature with PyVista

Set `MESH_FILE` to the first augmented file (the one that includes the mesh).
If you want to plot a later timestep, set `FIELD_FILE` to that file and keep `MESH_FILE`
pointing to the first file.


In [1]:
from pathlib import Path
import numpy as np
import pyvista as pv
from mpi4py import MPI

from pysemtools.datatypes.msh import Mesh
from pysemtools.datatypes.field import FieldRegistry
from pysemtools.io.ppymech.neksuite import pynekread


In [2]:
PROJECT_ROOT = Path.cwd()
MESH_FILE = PROJECT_ROOT / "data/processed_nek/phi0.40/h400x025_ref/premix_aug0.f00249"
FIELD_FILE = MESH_FILE  # change to another timestep if needed

if not MESH_FILE.exists():
    raise FileNotFoundError(MESH_FILE)
if not FIELD_FILE.exists():
    raise FileNotFoundError(FIELD_FILE)


In [3]:
comm = MPI.COMM_SELF
msh = Mesh(comm, create_connectivity=False)
fld = FieldRegistry(comm)

pynekread(str(MESH_FILE), comm, msh=msh)
pynekread(str(FIELD_FILE), comm, msh=msh, fld=fld, overwrite_fld=True)


2026-01-25 10:31:50,753  pynekread             INFO      Reading file: /media/alexandros/OS/Users/alexp/Documents/Bachelor Thesis/Code/data/processed_nek/phi0.40/h400x025_ref/premix_aug0.f00249
2026-01-25 10:31:50,766  Mesh                  INFO      Mesh object initialized from coordinates with type: float64 - Elapsed time: 0.007168333s
2026-01-25 10:31:50,767  pynekread             INFO      File successfully read - Elapsed time: 0.013757708s
2026-01-25 10:31:50,767  pynekread             INFO      Reading file: /media/alexandros/OS/Users/alexp/Documents/Bachelor Thesis/Code/data/processed_nek/phi0.40/h400x025_ref/premix_aug0.f00249
2026-01-25 10:31:50,778  Mesh                  INFO      Mesh object initialized from coordinates with type: float64 - Elapsed time: 0.007298405000000001s
2026-01-25 10:31:50,778  pynekread             INFO      Overwriting fld object
2026-01-25 10:31:50,798  Field                 INFO      Field registry updated with: ['u', 'v', 'p', 't', 's0', 's1', 's2

In [4]:
def build_pyvista_grid_2d(msh):
    x = np.asarray(msh.x)
    y = np.asarray(msh.y)
    z = np.asarray(msh.z)

    nelv, nz, ny, nx = x.shape
    if nz != 1:
        raise ValueError(f"Expected nz=1 for 2D; got nz={nz}")

    x2 = x[:, 0, :, :]
    y2 = y[:, 0, :, :]
    z2 = z[:, 0, :, :]

    points = np.column_stack([
        x2.ravel(order="C"),
        y2.ravel(order="C"),
        z2.ravel(order="C"),
    ])

    def idx(e, j, i):
        return ((e * ny + j) * nx + i)

    cells = []
    cell_types = []
    for e in range(nelv):
        for j in range(ny - 1):
            for i in range(nx - 1):
                n0 = idx(e, j, i)
                n1 = idx(e, j, i + 1)
                n2 = idx(e, j + 1, i + 1)
                n3 = idx(e, j + 1, i)
                cells.extend([4, n0, n1, n2, n3])
                cell_types.append(pv.CellType.QUAD)

    if not cells:
        raise RuntimeError("UnstructuredGrid has no cells – check mesh dimensions.")

    grid = pv.UnstructuredGrid(
        np.asarray(cells, dtype=np.int64),
        np.asarray(cell_types, dtype=np.uint8),
        points,
    )
    return grid


In [5]:
grid = build_pyvista_grid_2d(msh)
T = np.asarray(fld.fields["temp"][0])
grid.point_data["T"] = T.ravel(order="C")


In [6]:
pv.set_plot_theme("document")
plotter = pv.Plotter()
plotter.add_mesh(grid, scalars="T", cmap="inferno")
plotter.add_axes()
plotter.show()


Widget(value='<iframe src="http://localhost:40231/index.html?ui=P_0x76086d421d60_0&reconnect=auto" class="pyvi…